# Package 설치

In [ ]:
%pip install transformers tokenizers datasets accelerate sentencepiece pillow  timm -qU

Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
          exec(compile('''
          ~~~~^^^^^^^^^^^^
          # This is <pip-setuptools-caller> -- a caller that pip uses to run setup.py
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
          ...<32 lines>...
          exec(compile(setup_py_code, filename, "exec"))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
          ''' % ('C:\\Users\\FORYOUCOM\\AppData\\Local\\Temp\\pip-install-z4in16yl\\sentencepiece_f78d1e68e08843b09d4629612029e17e\\setup.py',), "<pip-setuptools-caller>", "exec"))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "<pip-setuptools-caller>", line 

In [16]:
import os
gpu = os.environ.get('CUDA_VISIBLE_DEVICES', '0')
# 현제 gpu 설정을 확인합니다.
if gpu == '':
    gpu = '0'

# 현재 GPU 설정과 cuda 버전을 출력합니다.
import torch
print(f"Using GPU: {gpu}")
print(f"CUDA Version: {torch.version.cuda}")

Using GPU: 0
CUDA Version: 12.8


# Hugging Face Pipeline을 이용한 모델 활용

- Pipeline은 Transformers 라이브러리의 가장 기본적인 객체로, **전처리 - 추론 -> 후처리** 로 이어지는 일련의 과정을 자동화하여 손쉽게 모델을 사용할 수 있게 해준다.
- Task에 따라 다양한 Pipeline 클래스를 제공하며 `pipeline` 함수를 이용해 쉽게 생성할 수 있다.
- **task만 지정**해서 기본 제공 모델과 토크나이저를 사용하거나 **직접 모델과 토크나이저를 지정**해 생성할 수 있다.
- https://huggingface.co/docs/transformers/pipeline_tutorial

![huggingface_pipeline.png](figures/huggingface_pipeline.png)

## 지원하는 주요 태스크
- https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline.task
### 자연어 처리 태스크
- **text-classification**: 텍스트 분류
- **text-generation**: 텍스트 생성
- **translation**: 번역
- **summarization**: 요약
- **question-answering**: 질의응답
- **fill-mask**: 마스크 토큰 채우기
- **token-classification**: 개체명 인식, Pos tagging 같이 개별 토큰에 대한 분류
- **feature-extraction**: 특징 추출(context vector)

### 영상 처리 태스크
- **image-classification**: 이미지 분류
- **object-detection**
  -  객체 검출 (Object Detection)
  -  이미지 안에서 객체들의 위치와 class를 찾아내는 작업
- **image-segmentation**
  -  이미지 세분화 (Image Segmentation)
  -  이미지를 픽셀 단위로 분할하여 각 픽셀이 어떤 객체에 속하는지 분류하는 작업

## 모델 검색
![huggingface_model_search.png](figures/huggingface_model_search.png)



## pipeline 함수
- 주요파라미터
  - **task:** 수행하려는 작업의 유형을 문자열로 지정한다.
  - **model:**
    - 사용할 사전 학습된 모델의 이름 또는 경로를 지정한다. 
    - 모델이름(ID)은 `[모델소유자이름]/[모델이름]` 형식이다. Hugging Face에서 제공하는 모델의 경우는 `모델소유자이름`이 생략되어 있다. (ex: "google/gemma-2-2b", "gpt2")
    - 모델을 명시적으로 지정하지 않으면, **task에 맞는 기본 모델이 로드**된다.
  - **tokenizer:** 자연어 task에서 사용할 토크나이저를 지정한다. 생략하면 모델과 같이 제공되는(model과 이름이 같은 토크나이저) 토크나이저를 사용한다.
  - **framework:** 사용할 딥러닝 프레임워크를 지정한다. 'pt'는 PyTorch(Default), 'tf'는 TensorFlow를 지정한다.
  - **device:** Pipeline 모델을 실행할 디바이스를 지정한다. 문자열로 `"cpu", "cuda:1", "mps"`, 또는 GPU 번호를 정수로 지정한다. 
  - **revision:** 모델의 특정 버전을 지정할 때 사용한다.
  - **trust_remote_code:** hub 모델을 직접 다운 받는 것이 아니라 모델을 다운 받는 **코드**를 다운 받아 local에서 실행하는 경우 코드를 실행할 수있게 할 지 여부. (bool)
  - **use_fast:** 
    - 빠른 토크나이저를 사용할지 여부를 지정합니다. 기본값은 True입니다.
    - 빠른 토크나이저는 `Rust` 언어로 구현되어 속도가 빠르다. 단 모든 모델에 대해 지원하지 않는다. 지원하지 않을 경우 `use_fast=True`로 설정해도 일반 토크나이저가 사용된다.

## Task 별 pipeline 실습

### 텍스트 분류

In [70]:
from transformers import pipeline

In [7]:
# 모델과 토크나이저를 로딩해서 pipeline을 생성합니다.
pipe = pipeline(task='text-classification', framework='pt')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


In [10]:
result = pipe("I am very unhappy") # raw -> tokenization -> model inference -> post-processing
result

[{'label': 'NEGATIVE', 'score': 0.9997686743736267}]

In [12]:
data = [ 
    "The project was completed successfully.", 
    "She always brings positive energy to the team.", 
    "I am confident that we will achieve our goals.",
    "The results were not as expected.", 
    "He struggled to meet the deadline.", 
    "The client was dissatisfied with the final product." 
]
results = pipe(data)
results

[{'label': 'POSITIVE', 'score': 0.9998227953910828},
 {'label': 'POSITIVE', 'score': 0.9998812675476074},
 {'label': 'POSITIVE', 'score': 0.9998470544815063},
 {'label': 'NEGATIVE', 'score': 0.9978100657463074},
 {'label': 'NEGATIVE', 'score': 0.99960857629776},
 {'label': 'NEGATIVE', 'score': 0.9996129870414734}]

In [14]:
# 특정 모델을 사용
model="distilbert-base-uncased-finetuned-sst-2-english"
#huggingface.co/models 에서 모델을 검색할 수 있습니다.

pipe = pipeline(task='text-classification', model=model, tokenizer=model)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0


In [15]:
pipe(data)

[{'label': 'POSITIVE', 'score': 0.9998227953910828},
 {'label': 'POSITIVE', 'score': 0.9998812675476074},
 {'label': 'POSITIVE', 'score': 0.9998470544815063},
 {'label': 'NEGATIVE', 'score': 0.9978100657463074},
 {'label': 'NEGATIVE', 'score': 0.99960857629776},
 {'label': 'NEGATIVE', 'score': 0.9996129870414734}]

In [16]:
kor_texts = [
    "이 영화 정말 재미있어요!",
    "서비스가 별로였어요.",
    "제품 품질이 우수합니다.",
    "따듯하고 부드럽고 제품은 너무 좋습니다. 그런데 배송이 너무 늦네요."  # 애매한 것 0.56 정도 나오네.
]
pipe(kor_texts)

[{'label': 'POSITIVE', 'score': 0.9855567812919617},
 {'label': 'POSITIVE', 'score': 0.7425784468650818},
 {'label': 'POSITIVE', 'score': 0.6555735468864441},
 {'label': 'NEGATIVE', 'score': 0.5247911810874939}]

In [18]:
model = 'Copycats/koelectra-base-v3-generalized-sentiment-analysis' 
pipe = pipeline(task='text-classification', model=model, tokenizer=model)
pipe(kor_texts)

Device set to use cuda:0


[{'label': '1', 'score': 0.9897311329841614},
 {'label': '0', 'score': 0.9969298243522644},
 {'label': '1', 'score': 0.9640170335769653},
 {'label': '0', 'score': 0.5669124126434326}]

In [19]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")
pipe(kor_texts)

Device set to use cuda:0


[{'label': 'Very Positive', 'score': 0.5016761422157288},
 {'label': 'Negative', 'score': 0.5245441794395447},
 {'label': 'Very Positive', 'score': 0.6271341443061829},
 {'label': 'Very Positive', 'score': 0.5461600422859192}]

In [20]:
for txt, r in zip(data, results):
    print(f"{txt} -> {r['label']} ({r['score']:.2f})")

The project was completed successfully. -> POSITIVE (1.00)
She always brings positive energy to the team. -> POSITIVE (1.00)
I am confident that we will achieve our goals. -> POSITIVE (1.00)
The results were not as expected. -> NEGATIVE (1.00)
He struggled to meet the deadline. -> NEGATIVE (1.00)
The client was dissatisfied with the final product. -> NEGATIVE (1.00)


### 제로샷 분류
- 제로샷(Zero-shot)은 각 개별 작업에 대한 특정 교육 없이 작업을 수행할 수 있는 task다.
- 입력 텍스트와 함께 클래스 레이블을 제공하면 분류 작업을 한다.
- 모델은  `task`에서 `Zero-Shot` 으로 시작하는 task를 선택하여 검색한다.

In [23]:
model = "facebook/bart-large-mnli"

text = ["Python is a programming language.", 
        "I love soccer", 
        "The stock price rose slightly today."]
labels = ['IT', 'Sports']

In [26]:
pipe= pipeline(task="zero-shot-classification", model=model)
result = pipe(text, candidate_labels=labels)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0


In [27]:
result

[{'sequence': 'Python is a programming language.',
  'labels': ['IT', 'Sports'],
  'scores': [0.5758525729179382, 0.4241473972797394]},
 {'sequence': 'I love soccer',
  'labels': ['Sports', 'IT'],
  'scores': [0.993531346321106, 0.006468650884926319]},
 {'sequence': 'The stock price rose slightly today.',
  'labels': ['IT', 'Sports'],
  'scores': [0.6849518418312073, 0.31504809856414795]}]

In [28]:
labels = ['IT', 'Sports', 'Finance', 'Politics', 'Entertainment', 'movie', 'education']
result = pipe(text, candidate_labels=labels)
result

[{'sequence': 'Python is a programming language.',
  'labels': ['IT',
   'Finance',
   'Sports',
   'movie',
   'education',
   'Entertainment',
   'Politics'],
  'scores': [0.22944369912147522,
   0.16977952420711517,
   0.16899801790714264,
   0.12504810094833374,
   0.10754953324794769,
   0.10339551419019699,
   0.09578562527894974]},
 {'sequence': 'I love soccer',
  'labels': ['Sports',
   'Entertainment',
   'IT',
   'movie',
   'education',
   'Finance',
   'Politics'],
  'scores': [0.9748833179473877,
   0.013839487917721272,
   0.006347238086163998,
   0.0016157771460711956,
   0.0012041475856676698,
   0.0011728359386324883,
   0.0009371955529786646]},
 {'sequence': 'The stock price rose slightly today.',
  'labels': ['Finance',
   'IT',
   'Entertainment',
   'Sports',
   'movie',
   'Politics',
   'education'],
  'scores': [0.40594515204429626,
   0.2021813541650772,
   0.09645739197731018,
   0.09299463778734207,
   0.07514490187168121,
   0.07279049605131149,
   0.0544860

### 텍스트 생성

In [29]:
pipe = pipeline(task='text-generation')


No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0


In [30]:
start_text = "Today weather"
sent = pipe(start_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [32]:
print(sent[0]['generated_text'])

Today weather and precipitation is expected to rise.

"We're very confident that this will be a very hot and dry day," said Kim. "The heat will take an hour to cool down, and the extreme precipitation will be a bit more intense than it was last year."

The weather patterns have been expected to change with the start of the summer heat wave. The National Weather Service predicts that the drought in California will have a "very strong" impact on the state's precipitation totals as well as on the rest of the country.

"The precipitation will be particularly strong up to the southern portion of the state, when the snowpack will be very heavy and severe," said Kim. "This will be particularly intense in central and eastern parts of the state, and the rest of the country, as well."

The drought is expected to push precipitation totals to nearly 3 million tons of snow by the end of July. In the worst-case scenario, it could produce up to 10 million tons of snow by the end of August, which is a

In [33]:
pipe(["I am", "Python", "LLM is"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[[{'generated_text': "I am so grateful to see you now. I have been feeling even more exhausted since you left. I'm so thankful to have you, and you're always so kind and kind. Thank you, for your kind words and for your gentle words, for your encouragement and advice.\n\nI have been so sorry to see you. I was so sorry to see you so badly. I know that you are sorry for what you've done, but I ask you to forgive me for the pain you've caused me, for all the many things that I've done to you. I am so sorry for everything I've done to you, but I am so sorry for everything I am trying to do to you.\n\nI am so sorry for what I've done to you. As long as I was in your shoes, you would not have been able to forgive me, because of everything that you have done to me. I am so sorry for everything that I've done to you.\n\nI am so sorry for everything that I've done to you. As long as I was in your shoes, you would not have been able to forgive me, because of everything that you have done to me. 

In [35]:
pipe("나는 어제")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '나는 어제 시간 콙사도 혌드 모세성 / 것마리 목하고 사람 강에자 안리땄 젠야 그리하고 는할는 휄고 건서요 배노리세 다으는 정출 혼정도 모세성 목하고 는할 아리땄 젠야 그리하고 는할는 정출 혼정도 모세성 목하고 는할는 정�'}]

In [ ]:
model_id = 'Qwen/Qwen3-0.6B'
pipe = pipeline(task='text-generation', model=model_id, tokenizer=model_id, device=0)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the docum

[{'generated_text': '오늘 날씨는 추운하고 일어나는 날입니다. 이는 기온이 20도에서 25도로, 낮은 일기입니다. 이 날에는 캐리어를 했으며, 빠르게 일어나, 이어지는 방향으로 향하고 있습니다. 캐리어는 매우 빠르고, 낮은 향상, 전면에서의 캐리어가 매우 흔하고 있습니다. 이 일기에서, 나는 주변의 환경에 대한 영향을 미치는 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소들을 설명하고, 이 요소'}]

In [ ]:
start_text = "오늘 날씨는"
pipe(start_text, max_length=50, do_sample=True, temperature=0.7)


In [ ]:
msg = [
    {"role": "user", "content": "LLM에 대해 설명해줘"},
]
pipe(msg , max_length=100, do_sample=True, temperature=0.7)

[{'generated_text': [{'role': 'user', 'content': 'LLM에 대해 설명해줘'},
   {'role': 'assistant',
    'content': "<think>\nOkay, the user wants an explanation of LLMs. Let me start by recalling what an LLM is. It's a large language model, right? I should define it clearly. Maybe mention that it's a type of artificial intelligence model developed by companies like Google, Amazon, and Microsoft. Then, talk about its key features: it can understand and generate text, perform tasks like writing code, and even answer questions. I need to highlight some examples, like Google's BERT or Amazon's Qwen. Also, mention the training data and how it's trained. Oh, and the benefits and challenges, like bias and transparency. Make sure to keep the explanation simple and not too technical. Avoid jargon where possible and use clear examples. Check if the user might be a student or someone new to AI, so keep the language accessible. Alright, that should cover it.\n</think>\n\nLLM은 **대규모 자연어 처리 모델**(Large Langua

In [41]:
pipe(msg)

[{'generated_text': [{'role': 'user', 'content': 'LLM에 대해 설명해줘'},
   {'role': 'assistant',
    'content': "<think>\nOkay, the user wants an explanation of LLM. Let me start by defining LLM. I should mention it's a large language model, so I'll use the example of GPT. Then, explain its key features like understanding and generating text, handling multiple tasks, and adaptability. Need to highlight the benefits and limitations. Maybe mention it's a large model, so I should clarify that it's not a physical computer. Also, discuss the use cases and the current state of LLM development. Make sure the explanation is clear and covers all important points without getting too technical.\n</think>\n\n**LLM (Large Language Model)**는 매우 학습된 인공지능으로 구성된 모델로, 문법과 언어 기반의 다양한 작업에 대해 인식하고, 생성되는 문장을 생성하거나 분석하는 능력을 갖는다. 예를 들어, GPT는 문법을 이해하며, 다양한 질문에 대한 답변을 제공하고, 상황에 따라 다양한 작업을 수행하는 능력을 보여준다.\n\n### 주요 특징:\n1. **문법과 언어 이해력**:  \n   LLM은 문법을 깊이 이해하고, 다양한 문장 구조를 분"}]}]

### 마스크 채우기

In [42]:
text = "I'm going to <mask> because <mask> am hurt."
model="distilroberta-base"

In [55]:
pipe = pipeline(task="fill-mask", model=model)
result = pipe(text, top_k=2)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


PipelineException: No mask_token (<mask>) found on the input

In [50]:
result

[[{'score': 0.26520201563835144,
   'token': 8930,
   'token_str': ' cry',
   'sequence': "<s>I'm going to cry because<mask> am hurt.</s>"},
  {'score': 0.060890741646289825,
   'token': 3581,
   'token_str': ' sleep',
   'sequence': "<s>I'm going to sleep because<mask> am hurt.</s>"}],
 [{'score': 0.9930053353309631,
   'token': 38,
   'token_str': ' I',
   'sequence': "<s>I'm going to<mask> because I am hurt.</s>"},
  {'score': 0.006336236838251352,
   'token': 939,
   'token_str': ' i',
   'sequence': "<s>I'm going to<mask> because i am hurt.</s>"}]]

In [56]:
text = "오늘 밤은 전국이 흐린 가운데 대부분 지역에 [MASK]가 내리겠고, 기온이 내려가면서 점차 [MASK]이 오는 곳이 많겠습니다"
model='beomi/kcbert-base'
pipe = pipeline(task="fill-mask", model=model)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [58]:
pipe(text, top_k=2)

[[{'score': 0.6340433359146118,
   'token': 4072,
   'token_str': '##서',
   'sequence': '[CLS] 오늘 밤은 전국이 흐린 가운데 대부분 지역에서 가 내리겠고, 기온이 내려가면서 점차 [MASK] 이 오는 곳이 많겠습니다 [SEP]'},
  {'score': 0.11311666667461395,
   'token': 28206,
   'token_str': '비가',
   'sequence': '[CLS] 오늘 밤은 전국이 흐린 가운데 대부분 지역에 비가 가 내리겠고, 기온이 내려가면서 점차 [MASK] 이 오는 곳이 많겠습니다 [SEP]'}],
 [{'score': 0.10058385133743286,
   'token': 10108,
   'token_str': '바람',
   'sequence': '[CLS] 오늘 밤은 전국이 흐린 가운데 대부분 지역에 [MASK] 가 내리겠고, 기온이 내려가면서 점차 바람 이 오는 곳이 많겠습니다 [SEP]'},
  {'score': 0.049839794635772705,
   'token': 10741,
   'token_str': '영향',
   'sequence': '[CLS] 오늘 밤은 전국이 흐린 가운데 대부분 지역에 [MASK] 가 내리겠고, 기온이 내려가면서 점차 영향 이 오는 곳이 많겠습니다 [SEP]'}]]

### Token별 분류
- task: token-classification 
  - 개체명인식(ner), 품사부착(pos tagging)을 수행하는 task 
  - 개체명 인식은 문장에서 특정한 개체명(예: 사람 이름, 지명, 조직명 등)을 식별하는 task이다. 

In [60]:
text = "My name is Sylvain and I work at Hugging Face in Brooklyn."
model = "dbmdz/bert-large-cased-finetuned-conll03-english"

pipe = pipeline(task="token-classification", model=model)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [62]:
result = pipe(text)
result

[{'entity': 'I-PER',
  'score': np.float32(0.99938285),
  'index': 4,
  'word': 'S',
  'start': 11,
  'end': 12},
 {'entity': 'I-PER',
  'score': np.float32(0.99815494),
  'index': 5,
  'word': '##yl',
  'start': 12,
  'end': 14},
 {'entity': 'I-PER',
  'score': np.float32(0.9959072),
  'index': 6,
  'word': '##va',
  'start': 14,
  'end': 16},
 {'entity': 'I-PER',
  'score': np.float32(0.99923277),
  'index': 7,
  'word': '##in',
  'start': 16,
  'end': 18},
 {'entity': 'I-ORG',
  'score': np.float32(0.9738931),
  'index': 12,
  'word': 'Hu',
  'start': 33,
  'end': 35},
 {'entity': 'I-ORG',
  'score': np.float32(0.976115),
  'index': 13,
  'word': '##gging',
  'start': 35,
  'end': 40},
 {'entity': 'I-ORG',
  'score': np.float32(0.9887976),
  'index': 14,
  'word': 'Face',
  'start': 41,
  'end': 45},
 {'entity': 'I-LOC',
  'score': np.float32(0.9932106),
  'index': 16,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [ ]:
model = "facebook/bart-large-mnli"

text = ["Python is a programming language.", 
        "I love soccer", 
        "The stock price rose slightly today."]
labels = ['IT', 'Sports']

### 질의 응답
- 문서와 질문을 주면 문서에서 답을 찾아 응답한다.

In [63]:
model = "distilbert-base-cased-distilled-squad"
pipe = pipeline(task='question-answering', model=model)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0


In [66]:

question="Where do I work?"
question="Where is Hugging Face?"
context="My name is Sylvain and I work at Hugging Face in Brooklyn"

result = pipe(question=question, context=context)
result 

{'score': 0.9893267154693604, 'start': 49, 'end': 57, 'answer': 'Brooklyn'}

In [67]:
context = """우리나라 2대 수출 품목인 자동차가 도널드 트럼프 미국 행정부의 관세 여파로 지난달 큰 폭의 수출 감소율을 보이면서 우려가 커지고 있다. 현대차, 기아의 미국 수출 비중이 최대 85%에 이르는 상황에서 자동차 관세 장기화 시 피해는 걷잡을 수 없이 불어날 것이라는 암울한 전망이 나온다.
1일 산업통상자원부가 발표한 5월 수출입 동향에 따르면 지난달 자동차 수출은 작년 동기 대비 4.4% 감소한 62억달러로 집계됐다. 최대 자동차 시장인 미국으로의 수출은 18억4000만달러로 무려 32.0% 급감했다.
4월 미국의 수입산 자동차 25% 관세 부과에 이어 5월부터 일부 자동차 부품에도 25%의 관세가 적용된 결과다. 관세 장기화 시 피해는 더 커질 것이라는 우려가 현실화한 셈이다.
국내 완성차 1·2위 업체인 현대차·기아는 현지 생산 비중을 확대하는 동시에 가격 인상을 검토하고 있다. 관세 여파를 흡수하기 위해서다. 가격 인상이 현실화할 경우 미국 현지 판매는 줄어들 수밖에 없어 수출에는 더 악영향을 미칠 것으로 보인다.
"""

q1 = "현대차 기아의 미국 수출비중은?"
q2 = "자동차 수출이 얼마나 급감했나?"
q3 = "대미 수출 감소에 국내 자동차 업체들의 대응방법은?"

In [68]:
model = "ainize/klue-bert-base-mrc"
pipe = pipeline(task='question-answering', model=model)

Device set to use cuda:0


In [71]:
result = pipe(question=[q1,q2,q3], context=context)
result

[{'score': 0.6396181583404541, 'start': 99, 'end': 102, 'answer': '85%'},
 {'score': 0.6323583126068115, 'start': 271, 'end': 276, 'answer': '32.0%'},
 {'score': 0.013693591579794884, 'start': 427, 'end': 433, 'answer': '가격 인상을'}]

### 문서 요약

In [ ]:
from transformers import pipeline
model = "eenzeenee/t5-base-korean-summarization"
pipe = pipeline(task='summarization', model=model)

### 번역

In [ ]:
model = "Helsinki-NLP/opus-mt-fr-en"
text = "Ce cours est produit par Hugging Face."

In [ ]:
model = "Helsinki-NLP/opus-mt-ko-en"

### 이미지를 설명하는 텍스트 생성

In [ ]:
url = "https://huggingface.co/datasets/Narsil/image_dummy/resolve/main/parrots.png"
url = "https://th.bing.com/th?id=ORMS.c526884bbea37c0bb9501f4f83b601e4&pid=Wdp&w=268&h=140&qlt=90&c=1&rs=1&dpr=1&p=0"
url = "http://images.cocodataset.org/val2017/000000039769.jpg"

In [ ]:
model = "ydshieh/vit-gpt2-coco-en"

### 이미지 분류

In [ ]:
url = "https://pds.joongang.co.kr/news/component/htmlphoto_mmdata/202306/25/488f9638-800c-4bac-ad65-82877fbff79b.jpg"

In [ ]:
model = "google/vit-base-patch16-224"

### Object Detection

In [ ]:
image_path = r"data/image1.jpg"
image_path = r"data/image2.jpg"
image_path = r"data/image3.jpg"

model='facebook/detr-resnet-50'